# RD Tech DPS5020 Power Supply Example

This is some demo code that goes along with [this video](https://youtu.be/BoTV7003glU).

It demonstrates some basics of using the Modbus protocol to control
an RD Tech DPS5020 power supply, version 1.2. It might work with later versions
or other products from them - I'm publishing the code so you can try it out and see!
If you're familar with Jupyter Notebooks, you can download and run this file as-is.
Otherwise, copy and paste what you want out of the GitHub page into a script or python terminal and 
play around with it that way.

My thought is I should be able to hack something like [Pronterface](https://github.com/kliment/Printrun)
to monitor the current/voltage output of ths supply while performing
[electrochemical machining](https://en.wikipedia.org/wiki/Electrochemical_machining),
and then have it step through the G-code to maintain a consistent feed rate and kerf width.
Since the resistance that the power supply sees is going to be dependent on the gap, 
that will affect the output voltage, similar to what you get with a [plasma cutter torch
height controller](https://www.youtube.com/watch?v=ZAWTuLad43g). The ability monitor the
cutting process would be a major improvement over the proof of concept shown by YouTube user
VanDro in [this video](https://www.youtube.com/watch?v=DINs7eMw7IQ).

You'll need the `minimalmodbus` Python package, which you can install with `sudo pip install minimalmodbus`
on most any Linux system. If your machine is having trouble finding
the power supply in `/dev/ttyUSB*` or `/dev/ttyACM*`, see the video for details and try installing the
[Arduino IDE](https://www.arduino.cc/en/Guide). If you're using Mac or Windows, it's *probably*
going to appear at `/dev/tty.usbmodem*` or `COM*`, respectively.


My test device was an electromagnet that was rated for 24 volts and ~0.5 amps.
The intput was from a generic 24 volt DC power supply.

[Here's](https://www.aliexpress.com/item/32821185351.html) the item on AliExpress, currently
under $70 all-in, and [here](https://drive.google.com/open?id=1nP9Vi6cgzv2zbxJG006256rYkz-R_jBL)
is where I found the documentation. (Yes, RD Tech publishes it on their Google Drive.)
[Here's](https://rdtech.aliexpress.com/store/923042) their official store for all their other products. 
I'd recommend only buying from the official store, since knock-offs are pretty common when 
ordering from Chinese export websites.

For more info on this supply, see [this teardown](https://www.youtube.com/watch?v=Q2rvAoO-MIA) 
from the EEVBlog and his [repair video](https://www.youtube.com/watch?v=VwdnGbI5ls8). (Note 
that the issue with ceramic capacitor has been fixed.) Also, I have an coming video with some
tips on assembling it easier. (TODO: add that link when published)

In [ ]:
import minimalmodbus
conn = minimalmodbus.Instrument('/dev/ttyUSB0', 1, debug=False)
conn.serial.timeout=1
conn.serial.baudrate=9600

In [ ]:
# Check that the connection looks sane.
conn.serial

In [ ]:
# Register values copied from the PDF documentation.
voltset = 0x00
ampset = 0x01
voltout = 0x02
ampout = 0x03
cvcc = 0x08
switch = 0x09

In [ ]:
# Values are fixed-point with two decimal places, so this is 12V, 0.5A.
conn.write_register(voltset, 1200)
conn.write_register(ampset, 50)

In [ ]:
# Confirming the settings were accepted.
conn.read_register(voltset)

In [ ]:
conn.read_register(ampset)

In [ ]:
# Powering the supply on.
conn.write_register(switch, 1)

In [ ]:
# Checking both the output voltage and output amperage.
# Note that this is read_registers (plural!), so it can read
# register slot 0x02 and 0x03 together, since they are next to each other.
conn.read_registers(voltout, 2)

In [ ]:
# Check that it should be in constant voltage mode, ie. 0.
conn.read_register(cvcc)

In [ ]:
# Drop the amperate limit to ~1/2 of what I'm seeing my device draw.
conn.write_register(ampset, 10)

# Note that since the output takes a moment to stabilize, reading
# it immediately will likely give bad results.
conn.read_registers(voltout, 2)

In [ ]:
# Reading again gives you stable numbers.
conn.read_registers(voltout, 2)

In [ ]:
# Now it'll be in constant amperage (ie. 1), and the output voltage will
# naturally drop by half.
conn.read_register(cvcc)

In [ ]:
# Trying to set the output voltage to the same as the input fails entirely.
# The power supply just ignores the request. It will only allow you to set it
# as high as V_in - 1. Since my supply is ~24.36V, my max setting is 23.35V.
conn.write_register(voltset, 2400)
conn.read_register(voltset)

In [ ]:
# This one works.
conn.write_register(voltset, 2300)
conn.read_register(voltset)

In [ ]:
# Checking the input voltage programatically.
# So in the general case, you could detect what your
# max should be in the code.
conn.read_register(0x05)

In [ ]:
# Dropping the amp max to 0 doesn't actually turn it off.
# You'll still see some residual voltage and amperage draw.
conn.write_register(ampset, 0)

In [ ]:
conn.read_register(cvcc)

In [ ]:
conn.read_registers(voltout, 2)

In [ ]:
# Jacking the amerage limit way up now.
# Since my input supply is 24V, and the DPS2050 is a buck converter,
# the maximum output voltage ends up being ~21.35V, even though
# the supply does let you set a higher voltage, as above.
conn.write_register(ampset,100)

In [ ]:
conn.read_registers(voltout, 2)

In [ ]:
# Now it still claims constant current, even though that's not exactly true.
conn.read_register(cvcc)

In [ ]:
# Turning it off.
conn.write_register(switch, 0)